# 동두천시 (1189)
- 작성자 : 박선빈
- 작성일자 : 2020년 10월 14일, 22일 수정

In [1]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
from pathlib import Path
import numpy as np
import re
from tqdm import tqdm

### 1. Raw File 불러오기

In [2]:
dat ="2020-10-22" 
df = pd.read_csv(Path.cwd().parent / (dat.replace('-', '')[2:] + "_type1.csv"))
soup = BeautifulSoup(df[df.num == 1189].html.iloc[0], 'html.parser')

### 2. 간단한 전처리 하기

In [3]:
rows = soup.select('div.layer')[2:]
rows1 = soup.select('div.titlebox>ul')
rows2 = soup.select('table.corona_table>tbody')      #동선 알려진 환자

### 3. 저장할 DataFrame 미리 만들기

In [8]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt'] 
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
#번호/날짜/경로/증상여부(있으면 1)/증상발현일/확진일/회복일?/격리시설/완치여부(퇴원=1)/전파자
#번호/순서/날짜/ 주소/장소명/ 상호명/시간/마스크/소독완료/이동수단

patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [5]:
fin= [rows.index(x) for x in rows if '삭제합니다' in str(x)][0]

In [9]:
start=0
for i in range(int(fin)):
    title = rows[i].select('p')
    where= rows1[i].select('p')
    
    number=where[0].text.split('#')[1]
    num = '1189' + '0' * (5- len(re.findall("\d+", number)[0])) + re.findall("\d+", number)[0]
    
    if '접촉자' in str(title[5]):
        pth= title[5].text.split(' 접촉자')[0].replace("○ ","").replace("의","")
    else: pth=''
    adt= title[1].text.split(': ')[1]
    if ('없음' or '무증상') in str(title[2]): 
        syy = 0
    else: 
        syy = 1
        
    if '퇴원' in str(where[3]):
        dch=1; atn= '퇴원'
    else:
        dch=0; atn = where[3].text
    cnd = title[3].text.split(':')[1].split('2020. ')[1]
    syd = ''; rsd =''; age = ''; sex = '';
    
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt)], columns = pat_col)
    patient = pd.concat([patient, new_pat])
    
    
    if '소독완료' in rows[i].text:
        aroute=rows2[start].select('tr>td')
        broute= [aroute.index(x) for x in aroute if '방문자' in str(x)]
        
        for j in range(len(broute)):
            rgt=aroute[5*j].text; frm=aroute[5*j+1].text; rgl=aroute[5*j+2].text; exd=aroute[5*j+3].text; ste=aroute[5*j+4].text; 
            ord= j+1
            new_route = pd.DataFrame(columns = rou_col, data = [[num, ord, dat, rgl, rgt, frm, exd, '', ste, '']])
            route = pd.concat([route, new_route])
        start+=1
        
    else:
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])
        route = pd.concat([route, new_route])

### 5. 결과 잘 나왔나 확인하기

In [10]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt
0,118900043,2020-10-22,,1,,10. 19.(월),,고양 생활치료센터,0,7명(검사결과:음성)
0,118900042,2020-10-22,동두천시#39,0,,10. 18.(일),,고양 생활치료센터,0,12명(검사결과:음성)
0,118900041,2020-10-22,동두천시#27,0,,10. 17.(토),,고양 생활치료센터,0,접촉자 없음(자가격리 중 확진)
0,118900040,2020-10-22,동두천시#35,1,,10. 16.(금),,고양 생활치료센터,0,33명(검사결과:음성)
0,118900039,2020-10-22,,1,,10. 16.(금),,경기도의료원 포천병원,0,"5명(양성 1명-동두천시#42),3명 타시 이관"
0,118900038,2020-10-22,동두천시#35,0,,10. 15.(목),,고양 생활치료센터,0,9명(검사결과:음성)
0,118900037,2020-10-22,동두천시#27,1,,10. 15.(목),,성남시의료원,0,"3명(동두천#27,#33,#41)"
0,118900036,2020-10-22,,1,,10. 12.(월),,고양 생활치료센터,0,전원 음성
0,118900035,2020-10-22,동두천시#32,0,,10. 12.(월),,고양 생활치료센터,0,"18명 /양성 2명(동두천시#38, #40)"
0,118900034,2020-10-22,동두천시#32,1,,10. 12.(월),,고양 생활치료센터,0,10명(검사결과:음성)


In [11]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,118900043,1,2020-10-22,,,,,,,
0,118900042,1,2020-10-22,평화로2544번길 45(보산동),일반미용업,은혜미용실,10. 14.(수) ~ 10. 16.(금),,소독완료,
0,118900041,1,2020-10-22,,,,,,,
0,118900040,1,2020-10-22,,,,,,,
0,118900039,1,2020-10-22,,,,,,,
0,118900038,1,2020-10-22,,,,,,,
0,118900037,1,2020-10-22,,,,,,,
0,118900036,1,2020-10-22,,,,,,,
0,118900035,1,2020-10-22,,,,,,,
0,118900034,1,2020-10-22,,,,,,,
